In [ ]:
#Installationen

# ! -> in Shell ausgeführt
#LangChain ist ein Python-Frameworkdas die Arbeit mit LLMseinfacher und strukturierter macht.
#LangChain-OpenAI-Integrationspaket für OpenAI installieren, um OpenAI-Modelle direkt in LangChain zu nutzen.
#LangChain-Community für die Integrationen zu externen Tools, Datenbanken, APIs, Dateiformaten.
#Reportlab ist ein Python-Paket, das PDFs erzeugen kann.
#requests ist ein HTTP-Paket für Python, um Web-Requests zu senden.
!pip install langchain langchain-openai langchain-community reportlab requests langchain_experimental

In [2]:
#Importe

#Os ist eine Standardbibliothek von Python, die als Betriebssystem-Funktionen dient.
#json ist eine Standardbibliothek von Python, die zwischen Python-Datenstrukturen und JSON umwandelt.
#requests ist ein externes Paket (vorher installiert), das für HTTP-Anfragen verwendet wird.
import os, json, requests

#ChatOpenAI ist die Schnittstelle zu den OpenAI-Chatmodellen.
from langchain_openai import ChatOpenAI

#Tools istallieren, damit Agent externe Funktionen/Skills zur Lösung von Aufgaben nutzen kann.
from langchain.agents import Tool

#Load_agent_executor erstellt einen Executor-Agent, der die vom Planner erzeugten Schritte ausführt und die entsprechenden Tools aufruft.
#Load_chat_planner erstellt einen Planner-Agent, der die User-Anfrage in einen Schritt-für-Schritt-Plan zerlegt.
#PlanAndExecute ist ein Agent-Wrapper und kombiniert Planner und Executer, um eine dynamische Orchestrierung zu bauen.
from langchain_experimental.plan_and_execute import load_agent_executor, load_chat_planner, PlanAndExecute

#SimpleDocTemplate stellt das "Grundgerüst" für ein PDF-Dokument.
#Paragraph unterstützt Formatierung, Schriftart, Größe, Stil und ermöglicht Textabsätze.
#Spacer fügt Abstand zwischen den Elementen hinzu, um das Layout sauber zu halten
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer

#Hilfsfunktion, um vorgefertigte Textstile (Stylesheet) für PDFs zu bekommen.
from reportlab.lib.styles import getSampleStyleSheet

#Ermöglicht Zugriff auf API-Key
from google.colab import userdata

In [3]:
#API Key laden
#OpenAI-API-Schlüssel sicher aus gespeicherten Colab-Benutzerdaten in Notebook schreiben

OPENAI_API_KEY = userdata.get('apikey_11')

#LLM
#ChatOpenAI --> Schnittstelle von LangChain zu OpenAI
#openai_api_key --> Schlüssel, damit LangChain auf Modell zugreifen kann
#model ist das Sprachmodell das bei OpenAI verwendet wird
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o-mini")

In [4]:
#Tools

#Funktionsdefinition für Rechnertool
#expression: str --> String als Input erwartet
#-> str --> gibt String zurück
def rechner_tool(expression: str) -> str:
    try:
        #eval(expression) --> wertet Rechenaufgabe als Python-Ausdruck aus
        #str() --> macht wieder zurückzugebenden String draus
        return str(eval(expression))
    #Exception-handeling --> gibt Fehler zurück, sodass Agent nicht abstürzt
    except Exception as e:
        return f"Fehler: {str(e)}"

#Funktionsdefinition für Wettertool
#city: str --> String als Input erwartet (Stadt)
#-> str --> gibt String zurück
def wetter_tool(city: str) -> str:
    try:
        #URL, wo das Wetterabgefragt wird
        url = f"https://wttr.in/{city}?format=j1"
        #Wandelt den Inhalt der URL in ein JSON-Format um
        resp = requests.get(url).json()
        #Speichert die Temperatur aus der JSON
        temp = resp["current_condition"][0]["temp_C"]
        #Speichert die Wetter-Konditionen aus der JSON
        desc = resp["current_condition"][0]["weatherDesc"][0]["value"]
        #Gibt Wetter zurück als string
        return f"In {city} sind es {temp}°C und {desc}."
    #Exception-handeling --> gibt Fehler zurück, sodass Agent nicht abstürzt
    except Exception as e:
        return f"Wetter-Fehler: {str(e)}"

#Liste mit Tool-Objekten erstellt. Enthält alle Tools, die der Agent nutzen kann
#Jedes Tool hat einen eindeutigen Namen, Funktion (vorher definiert), und Beschreibung.
#Die Beschreibung beschreibt dem Agenten, wann er das Tool einsetzen soll.
tools = [
    Tool(
        name="Rechner",
        func=rechner_tool,
        description="Berechne einen mathematischen Ausdruck. Input: Mathe-Ausdruck."
    ),
    Tool(
        name="Wetter",
        func=wetter_tool,
        description="Hole das Wetter einer Stadt. Input: Stadtname."
    )
]

In [5]:
#Planner und Executor
#Planner-Agent wird erstellt.
#Planner greift auf das llm (vorher definiert) zu.
#Überlegt, welche Schritt enotwendig sind und gibt Aufgaben in planner-Objekt zurück.
planner = load_chat_planner(llm)

#Executor-Agent wird erstellt, der Schritte ausführen kann.
#llm wird zur Textgenerierung und Interpretation genutzt.
#Tools-Liste, für zur verfügung stehende Tools.
#verbose=True --> es sollen logs angezeigt werden, welche Tools genutzt wurden
#executor-Objekt speichert und kann Schritte automatisch ausführen und Ergebnisse zurückgeben
executor = load_agent_executor(llm, tools, verbose=True)

In [6]:
#dynamischer Orchestrator (Plan and Execute)
#Kombiniert vorher definierte planner und executor. Kann dynamische Aufgaben planen und ausführen.
#verbose=True --> es sollen logs angezeigt werden, welche Tools genutzt wurden
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [ ]:
#Beispiel-Abfragen
#query ist die Nutzer-Abfrage
query = "Ich möchte die aktuelle Temperatur in Dresden durch 2 dividieren."
#result ist das Ergebnis. Der Agent wird mit der query aufgerufen. Durch die dynamische Orchestrierung, plant und führt der Agent die Schritte alleine aus.
result = agent.run(query)

print("Finale Antwort:", result)

In [ ]:
#Export als JSON & PDF

#Funktion export_output definieren
#In result soll der Ergebnis-Text gespeichert werden
#Die Datei soll "Ergebnis heißen"
def export_output(result: str, filename: str = "Ergebnis"):
    #data sind die zu speichernden Daten
    data = {"Ergebnis": result}
    #f ist die Referenz auf die Datei und "w" öffnet die Datei zum Schreiben
    #Wenn die Datei nicht extistiert, wird sie neu angelegt
    #Sonst wird die existierende Datei überschrieben
    #'with' Schließt die Datei automatisch, sobald der Block fertig ist
    with open(f"{filename}.json", "w") as f:
        #Hier wird die Datei mit den Daten beschrieben.
        #f gibt die Referenz.
        #indent=2 beschreibt die Formatierung (2 Leerzeichen pro Ebene). -> schöner Formatiert
        json.dump(data, f, indent=2)

    #Standardstil für PDF-Texte aus Paket.
    styles = getSampleStyleSheet()
    #Hier wird das PDF-Dokument erstellt.
    doc = SimpleDocTemplate(f"{filename}.pdf")
    #Liste der Elemente, die ins PDF kommen
    #Paragraph -> eigentlicher Ergebnistext
    #Paragraph("Ergebnis:", styles["Heading2"]) --> Überschrift
    #Spacer(1, 12) --> Abstand zwischen Überschrift und Text
    #Paragraph(result, styles["Normal"]) --> Text (Inhalt)
    story = [Paragraph("Ergebnis:", styles["Heading2"]), Spacer(1, 12),
             Paragraph(result, styles["Normal"])]
    #Hier wird das PDF gebaut.
    doc.build(story)

    #Als Kontrolle die Ausgabe für die gespeicherten Dokumente
    print(f"Gespeichert: {filename}.json und {filename}.pdf")

#Hier wird die Funktion aufgerufen
export_output(result, filename="Ergebnis")